In [1]:
import bs4 as bs
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as pplt
import os
from pptx import Presentation
import pprint as p

#Die Buch-Kürzel des alten Testaments
AT = ["Gn", "Ex", "Lv", "Nm", "Dt", "Ios", "Idc", "Rt", "1Sm", "2Sm", "3Rg", "4Rg", "1Par", "2Par", "Esr", "Neh", "Tb",
      "Idt", "Est", "1Mcc"
    , "2Mcc", "3Mcc", "4Mcc", "Iob", "Ps", "Prv", "Ecl", "Ct", "Sap", "Sir", "Is", "Ier", "Lam", "Bar", "Ez", "Dn",
      "Os", "Ioel", "Am", "Abd"
    , "Ion", "Mi", "Na", "Hab", "So", "Agg", "Za", "Mal"]


  
def percent_calc(num1, num2):
    '''returns percentage value'''
    return (num1/num2)*100
    
 
    
def analyse(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, this function analyses the chapter with regard to our criteria'''
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read()   
    soup = bs.BeautifulSoup(file, 'lxml')
    Refs = soup.select('ref[cRef]')
    # Konvertierung der Daten in data. Dies ist die Grundlage für die Auswertungen
    data = []
    for ref in Refs:
        if ref.get('cref').split('_')[0] in AT:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
        else:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
    #print(data)
    # der eigentliche Dataframe wird gesetzt
    df_data = pd.DataFrame(data)
    # der Header wird gesetzt.
    df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



    # Hier wird eine Liste der wörtlichen Zitate erstellt
    qs = []
    for ref in Refs:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
            qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))



    stellen = []
    for i in range (0, len(df_data)):
        if df_data['zitat'][i] != 'not a direct quote':
            stellen.append(df_data['stelle'][i])
    
    # Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche 
    quote_len = 0
    for i in range(len(qs)):
        quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
        i+=1



    ges = bs.BeautifulSoup(file, 'lxml')

    ges_soup = ges.select('div')
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
    
    print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
    print(f"wörtliche Zitate: {len(stellen)}")
    #p.pprint(f"Liste d. wörtlichen Zitate:{stellen}")
    print("Gesamtlänge:" + str(total_len))
    print("Zitatlänge :"+str(quote_len))    
    print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))
    lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(stellen)}", f"Liste d. wörtl.: {stellen}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
    # Speichern der obigen Ausgabe harter Fakten in einer Text-Datei mit dem Titel der ausgewählten Datei. 
    with open(f'Visualisierungen/{file_title}', 'w') as f2:
        for line in lines:
            f2.write(line)
            f2.write("\n")
    # Ausgabe, Speichern und Schließen der einzelen Diagramme. 
    print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["atnt"].value_counts().plot(figsize=(8,8), kind="bar", legend=False, title=f"{filename}", color=["blue", "red"]))
    pplt.savefig(f'Visualisierungen/{file_title}_atnt.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["kuerzel"].value_counts()[:5].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename}  @cRef)", color="purple"))
    pplt.savefig(f'Visualisierungen/{file_title}_cref.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["stelle"].value_counts()[:10].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename} nach Stellen", color="red"))
    pplt.savefig(f'Visualisierungen/{file_title}_nach_stellen.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    
    

# Daten für das gesamte Interim

In [2]:
#Probe-Sachen
with open(f'interim2.xml', 'r') as f:
    file = f.read()
soup = bs.BeautifulSoup(file, 'lxml')
Refs = soup.select('ref[cRef]')

# Eine leere Liste wird erstellt und mit den Bibelstellen gefüllt. Hierbei wird sortiert, ob sie im Alten Testament stehen oder nicht.
# Ein zweites Auswahlkriterium ist die Frage, ob direkt vor dem Eltern-ELement des <ref type='biblical'> ein <q> steht, und somit ein direktes Zitat vorliegt.
data = []
for ref in Refs:
    if ref.get('cref').split('_')[0] in AT:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
    else:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
# der eigentliche Dataframe wird gesetzt
df_data = pd.DataFrame(data)
# der Header wird gesetzt.
df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



stellen = []
for i in range (0, len(df_data)):
    if df_data['zitat'][i] != 'not a direct quote':
        stellen.append(df_data['stelle'][i])
print(len(stellen))

# Hier wird eine Liste der wörtlichen Zitate erstellt
qs = []
for ref in Refs:
    if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
        qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))
        print(ref['cref'])


# Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche
quote_len = 0
for i in range(len(qs)):
    quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
    i+=1



ges = bs.BeautifulSoup(file, 'lxml')

ges_soup = ges.select('div')
total_len=0
for i in range(len(ges_soup)):
    total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))

lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(qs)}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
with open('Visualisierungen/interim.txt', 'w') as f2:
    for line in lines:
        f2.write(line)
        f2.write("\n")

print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
print(f"wörtliche Zitate: {len(qs)}")
print("Gesamtlänge:" + str(total_len))
print("Zitatlänge :"+str(quote_len))
print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))


print(df_data.groupby("atnt").count())




114
Sir_15,14
Rm_5,12
2Cor_5,19
Ps_51,12
1Cor_6,11
Tit_3,5-7
Rm_7,25
1Cor_1,30
1Io_2,1-2
Tit_2,12-13
Mt_3,2
Mc_1,15
1Cor_15,58
Hbr_6,10
Sir_18,22
Apc_22,11
Mt_19,17
Ps_118,108
Rm_12,5
Eph_5,25-27
Eph_4,15-16
1Cor_12,26
Mt_18,17
Eph_4,11
Tit_3,10-11
2Tim_2,20
1Cor_1,10
Ps_2,8
Mt_8,11
Act_1,8
Io_14,16-17
2Pt_1,20-21
1Cor_5,13
Act_15,28
Eph_4,11
Act_13,2
Io_21,17
Hbr_13,17
Tit_3,5
Mt_28,19
Gal_5,16
Lc_24,49
Io_14,26
Io_20,22-23
Io_20,21-23
Ps_18,13
Io_20,23
1Cor_10,17
Mt_26,26
Mc_14,22
Lc_22,19
1Cor_11,24
Lc_22,20
1Cor_11,25
1Cor_11,29-30
Mc_6,13
Iac_5,14-15
1Cor_14,33
1Tim_4,14
Io_20,21-23
Mc_16,15
Mt_28,19
Lc_22,19
1Cor_11,24-25
Mt_19,5
Gn_2,24
Mt_19,4-6
Mt_19,8-9
1Cor_7,10-11
Gn_2,24
Mt_19,5
1Cor_7,4
1Cor_7,2
Gn_2,24
1Tim_2,15
Hbr_13,4
Gn_2,23-14
1Tim_2,4
2Cor_5,19
Io_1,29
1Io_2,2
Hbr_10,14
Col_1,19-20
Eph_1,10
Is_63,3
Is_53,5
Ps_49,12-13
Hbr_10,14
Mt_26,26
Lc_22,19
1Cor_11,24
Hbr_10,14
Mal_1,10-11
Mt_26,26
Mc_14,22
Lc_22,19
1Cor_11,24
Mal_1,11
Mal_1,10
Ps_109,4
Hbr_5,6
Hbr_7,17
Hbr_7,

In [3]:
for i in range(1,27):
    if i < 10:
        print(f"analyse('Artikel/0{i}.xml')")
    else:
        print(f"analyse('Artikel/{i}.xml')")

analyse('Artikel/01.xml')
analyse('Artikel/02.xml')
analyse('Artikel/03.xml')
analyse('Artikel/04.xml')
analyse('Artikel/05.xml')
analyse('Artikel/06.xml')
analyse('Artikel/07.xml')
analyse('Artikel/08.xml')
analyse('Artikel/09.xml')
analyse('Artikel/10.xml')
analyse('Artikel/11.xml')
analyse('Artikel/12.xml')
analyse('Artikel/13.xml')
analyse('Artikel/14.xml')
analyse('Artikel/15.xml')
analyse('Artikel/16.xml')
analyse('Artikel/17.xml')
analyse('Artikel/18.xml')
analyse('Artikel/19.xml')
analyse('Artikel/20.xml')
analyse('Artikel/21.xml')
analyse('Artikel/22.xml')
analyse('Artikel/23.xml')
analyse('Artikel/24.xml')
analyse('Artikel/25.xml')
analyse('Artikel/26.xml')


In [4]:
analyse('Artikel/01.xml')
analyse('Artikel/02.xml')
analyse('Artikel/03.xml')
analyse('Artikel/04.xml')
analyse('Artikel/05.xml')
analyse('Artikel/06.xml')
analyse('Artikel/07.xml')
analyse('Artikel/08.xml')
analyse('Artikel/09.xml')
analyse('Artikel/10.xml')
analyse('Artikel/11.xml')
analyse('Artikel/12.xml')
analyse('Artikel/13.xml')
#analyse('Artikel/14.xml')
analyse('Artikel/15.xml')
analyse('Artikel/16.xml')
analyse('Artikel/17.xml')
analyse('Artikel/18.xml')
analyse('Artikel/19.xml')
analyse('Artikel/20.xml')
analyse('Artikel/21.xml')
analyse('Artikel/22.xml')
analyse('Artikel/23.xml')
analyse('Artikel/24.xml')
#analyse('Artikel/25.xml')
analyse('Artikel/26.xml')

Anzahl an erfassten Bibelstellen: 2
wörtliche Zitate: 1
Gesamtlänge:2538
Zitatlänge :45
Prozentualer Anteil: 1.773049645390071
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 14
wörtliche Zitate: 1
Gesamtlänge:2749
Zitatlänge :75
Prozentualer Anteil: 2.728264823572208
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 8
wörtliche Zitate: 1
Gesamtlänge:2009
Zitatlänge :173
Prozentualer Anteil: 8.6112493777999
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 13
wörtliche Zitate: 7
Gesamtlänge:4292
Zitatlänge :944
Prozentualer Anteil: 21.99440820130475
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


In [5]:
analyse('interim.xml')

Anzahl an erfassten Bibelstellen: 283
wörtliche Zitate: 114
Gesamtlänge:204608
Zitatlänge :9713
Prozentualer Anteil: 4.747126212073819
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
